# Import libraries

In [1]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

2.6.4
Found GPU at: /device:GPU:0


2022-11-11 15:40:10.966442: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 15:40:11.029206: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:11.146741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:11.147591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [2]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0


2022-11-11 15:40:13.558692: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:13.559698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:13.560364: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Load data

Solo questo dovrebbe essere diverso durante la challenge

In [4]:
dataset_dir = '/kaggle/input/homework1/training_data_final'
image_size = (96,96)

# Dataloader and Data Augmentation

We can fine tune this

In [5]:
# Create an instance of ImageDataGenerator with Data Augmentationonly for training
aug_train_data_gen = tfk.preprocessing.image.ImageDataGenerator(rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        rescale=1/255.,
                                        validation_split=0.2)

aug_validation_data_gen = tfk.preprocessing.image.ImageDataGenerator(
                                        rescale=1/255.,
                                        validation_split=0.2)

train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed,
                                               subset='training')

validation_gen = aug_validation_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=image_size,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed,
                                               subset='validation')

Found 2836 images belonging to 8 classes.
Found 706 images belonging to 8 classes.


In [6]:
print("Assigned labels")
print(train_gen.class_indices)
print()
print("Target classes")
print(train_gen.classes)
num_classes = train_gen.num_classes

Assigned labels
{'Species1': 0, 'Species2': 1, 'Species3': 2, 'Species4': 3, 'Species5': 4, 'Species6': 5, 'Species7': 6, 'Species8': 7}

Target classes
[0 0 0 ... 7 7 7]


# Load pre-trained model for transfer learning

We can fine tune this with different pretrained models

In [7]:
# Create the base model from the pre-trained model MobileNet V3
IMG_SHAPE = image_size + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

2022-11-11 15:40:14.570362: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:14.571310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:14.571985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:14.572718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-11 15:40:14.573383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

9420800/9406464 [==============================] - 0s 0us/step


In [8]:
base_model.summary()

Model: "mobilenetv2_1.00_96"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 48, 48, 32)   0           bn_Conv1[0][0]                   
________________________________________________________________________________

In [9]:
def build_model(input_shape, output_classes, learning_rate=0.001, freeze=True):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Freeze the base model
    if(freeze):
      base_model.trainable = False
    else:
      base_model.trainable = True
    # We need training=False for the BatchNormalization layer
    feature_extractor = base_model(input_layer , training=not freeze)

    x = tfkl.GlobalAveragePooling2D()(feature_extractor)
    x = tfkl.Dropout(0.2)(x)
    x = tfkl.Dense(units=256, activation='relu', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='hidden_layer_1')(x)
    x = tfkl.Dropout(0.2)(x)
    output_layer = tfkl.Dense(units=output_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate), metrics='accuracy')

    # Return the model
    return model

In [10]:
model = build_model(IMG_SHAPE, num_classes, learning_rate=0.005)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 96, 96, 3)]       0         
_________________________________________________________________
mobilenetv2_1.00_96 (Functio (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 256)               327936    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
output_layer (Dense)         (None, 8)                 2056  

In [11]:
history = model.fit(
        x = train_gen,
        epochs = 50,
        validation_data = validation_gen
    ).history

2022-11-11 15:40:17.773668: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-11-11 15:40:21.443433: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


355/355 [==============================] - 34s 69ms/step - loss: 1.7279 - accuracy: 0.4030 - val_loss: 1.2729 - val_accuracy: 0.5198
Epoch 2/50
355/355 [==============================] - 13s 36ms/step - loss: 1.4236 - accuracy: 0.4679 - val_loss: 1.3004 - val_accuracy: 0.5042
Epoch 3/50
355/355 [==============================] - 15s 41ms/step - loss: 1.3935 - accuracy: 0.4961 - val_loss: 1.2391 - val_accuracy: 0.5552
Epoch 4/50
355/355 [==============================] - 18s 49ms/step - loss: 1.3995 - accuracy: 0.4933 - val_loss: 1.2937 - val_accuracy: 0.5283
Epoch 5/50
355/355 [==============================] - 13s 36ms/step - loss: 1.3552 - accuracy: 0.4996 - val_loss: 1.2025 - val_accuracy: 0.5552
Epoch 6/50
355/355 [==============================] - 20s 56ms/step - loss: 1.3727 - accuracy: 0.5046 - val_loss: 1.1816 - val_accuracy: 0.5822
Epoch 7/50
355/355 [==============================] - 13s 36ms/step - loss: 1.3587 - accuracy: 0.5257 - val_loss: 1.3262 - val_accuracy: 0.5297
Epo

In [12]:
model.save("./models/test")

2022-11-11 15:52:37.308690: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
